# TP — Barre d’actions & animation d’attaque (Arcade)

**Objectifs**

- Afficher une **barre d’icônes** en bas de l’écran (sprites `assets/GUI`).
- Cliquer une icône → **mode `clicked`** pendant le **Cooldown** (lu dans `warrior_actions.json`).
- Lier le clic à l’**animation d’attaque** du joueur, puis retour à la marche.

**Pré-requis**

- `constants.py` contient : `GUI_ACTIONS_SPRITE`, `GUI_ACTIONS_SPRITE_DOWN`, `ACTIONBAR_SLOTS`, `ACTIONBAR_BINDINGS`, `PLAYER_ACTIONS_FILE`.
- `assets/Player/warrior_actions.json` existe (ex. `attack_0` avec un champ `"Cooldown"`).
- `timer.py` fourni.

> Ce notebook donne les **extraits à copier-coller** dans ton projet. Il n’exécute pas Arcade ici.

## Partie 1 — Barre d’icônes + cooldown `clicked`
On s’appuie **strictement** sur les constantes et fichiers existants.

### Étape 1 — Vérifier les constantes (`constants.py`)
Assure-toi que **ces constantes** existent (adapter seulement la casse/chemins si besoin) :

In [ ]:
# constants.py (extrait attendu)
GUI_ACTIONS_SPRITE      = "assets/GUI/actions_player.png"
GUI_ACTIONS_SPRITE_DOWN = "assets/GUI/actions_player_clicked.png"

ACTIONBAR_SLOTS = 6 # 6 actions pour l'instant

PLAYER_ACTIONS_FILE = "assets/Player/warrior_actions.json"


### Étape 2 — Charger les actions du joueur (`player.py`)
Dans le fichier `warrior_actions.json`, **écrire au moins deux autres actions** (attack_1 et attack_2 minimum).
Dans `__init__` du `Player`, déclare `action_name` et charge `warrior_actions.json` via la **constante** :

In [ ]:
# Ajouter dans __init__() -- class player.py (extrait)

class Player(...):
    def __init__(self, ...):
        ...
        # -- TP 2 : Ajout du niveau du joueur --
        self.level = 1
        # -- TP 2 : FIN --
        
        # -- TP 2 : Ajout des actions -- 
        self.actions = None     # Dictionnaire des actions
        self.action_name = ""   # Nom de l'action menée en cours (attaque ici)
        # -- TP 2 : FIN --

In [ ]:
# Modifier (et ajouter) dans setup() après super().setup() -- class player.py

# Ouverture des fichiers JSON
# -- TP 2 : Chargement des fichiers -- 
# Chargement des caractéristiques du joueur
with open(PLAYER_CARACTERISTICS_FILE, "r", encoding="utf-8") as f:
    data: dict = json.load(f)
self.attributes = data["Player"]

# Expérience initiale du joueur
self.xp = self.attributes["Experience"]
 
       
# Chargement des actions (fichier .json)
with open(PLAYER_ACTIONS_FILE, "r", encoding="utf-8") as f:
    self.actions: dict = js
# -- TP 2 : FIN --

### Étape 3 — GUI : chargement des icônes + construction de la barre (`gui.py`)

La *classe ActionButton* permet de **modéliser par une icône une action du joueur (attaque ici)**. 
On évite `load_texture(x, y, w, h)` (non dispo selon versions), et on découpe via **Pillow (PIL)**.
**Ajouter/compléter** ceci dans `gui.py` :

In [ ]:
# Au début de gui.py (extraits clés)
# -- TP 2 : Ajout d'import --
from PIL import Image
import tempfile, os
from timer import Timer
# -- TP 2 : FIN --


# Ajout après la class QuitButton -- class gui.py
# -- TP 2 : Ajout des icônes d'action --
class ActionButton(arcade.gui.UITextureButton):
    def set_window(self, game): self.game = game
    def set_name(self, text):   self.name = text  # ex: "attack_0"
    def on_click(self, event: arcade.gui.UIMousePressEvent):
        if self.game.gui.active:
            return

        self.game.player.status |= PLAYER_ATTACK # Mode attaque
        self.game.player.action_name = self.name
        self.game.player.current_texture_indice = 0
        self.game.player.attack_tick = 0
        self.game.player.attack(self.game.player.clicked_mob)
        self.game.gui.active = True
        self.game.gui.update()
        cd = float(self.game.player.actions[self.name]["Cooldown"])
        self.game.player_timer = Timer(cd)
        self.game.player_timer.start()
# -- TP 2 : FIN --

# Dans le --init--()  -- class gui.py
class Gui:
    def __init__(self, game):
        self.game = game
        if not hasattr(self.game, "manager") or self.game.manager is None:
            self.game.manager = arcade.gui.UIManager()
        self.game.manager.enable()

        self.anchor = None
        self.menu_bar = None
        self.player_box = None
        self.action_player_box = None
        self.clicked_mob_box = None
        self.active = False

        # -- TP 2 : Ajout des textures --
        self.actions_player_textures = []          # GUI active
        self.actions_player_textures_clicked = []  # GUI inactive
        self._tmp_icon_files = []    # Fichiers temporaires
        self.create_textures()       # Créations des textures
        # -- TP 2 : FIN --
    
    # -- TP 2 : Création des textures --
    # Méthode à ajouter
    # Partie technique, juste copier-coller
    def create_textures(self):
        # Attention aux coordonnées à adapter si nouvelles images
        rects = [(i * 32, 98, 32, 32) for i in range(ACTIONBAR_SLOTS)]
        
        img_normal  = Image.open(GUI_ACTIONS_SPRITE).convert("RGBA")
        img_clicked = Image.open(GUI_ACTIONS_SPRITE_DOWN).convert("RGBA")
        
        self.actions_player_textures.clear()
        self.actions_player_textures_clicked.clear()
        
        for i, (x, y, w, h) in enumerate(rects):
            crop = img_normal.crop((x, y, x + w, y + h))
            tmp = tempfile.NamedTemporaryFile(prefix=f"act_norm_{i}_", suffix=".png", delete=False)
            crop.save(tmp.name); tmp.close()
            self._tmp_icon_files.append(tmp.name)
            tex = arcade.load_texture(tmp.name)
            self.actions_player_textures.append(tex)
        for i, (x, y, w, h) in enumerate(rects):
            crop = img_clicked.crop((x, y, x + w, y + h))
            tmp = tempfile.NamedTemporaryFile(prefix=f"act_click_{i}_", suffix=".png", delete=False)
            crop.save(tmp.name); tmp.close()
            self._tmp_icon_files.append(tmp.name)
            tex = arcade.load_texture(tmp.name)
            self.actions_player_textures_clicked.append(tex)
    # -- TP 2 : FIN --

    # -- TP 2 : Création des icônes d'action --
    # Méthode à ajouter
    def create_action_box(self):
        self.action_player_box = arcade.gui.UIBoxLayout(vertical=False)
        textures = self.actions_player_textures_clicked if self.active else self.actions_player_textures
        player_level = int(self.game.player.attributes.get("Level", 1))
        for i in range(ACTIONBAR_SLOTS):
            action_name = ACTIONBAR_BINDINGS.get(i, f"attack_{i}")
            action_def = self.game.player.actions.get(action_name)
            if not action_def:
                continue
            if player_level < int(action_def.get("Level", 1)):
                continue
            if i >= len(textures):
                break
            btn = ActionButton(texture=textures[i], width=50, height=50)
            btn.set_window(self.game)
            btn.set_name(action_name)
            self.action_player_box.add(btn)
        # Modifier la place si on le souhaite :)
        self.anchor.add(child=self.action_player_box, anchor_x="left", anchor_y="bottom")
    # -- TP 2 : FIN --

    # A ajouter dans les méthodes setup() et add_clicked_mob() après le `box joueur` :)
    # Boite d'actions
    # -- TP 2 : Création des icones --
    self.create_action_box()
    # -- TP 2 : FIN --

**<u>Question</u>** : Pourquoi vaut-il mieux utiliser l'instruction *self.game.player.status |= PLAYER_ATTACK* que *self.game.player.status ^= PLAYER_ATTACK* ? On rappelle que | = OR bit à bit et ^ = XOR bit à bit.

### Étape 4 — Fin du cooldown (`on_update`)
Quand le `Timer` expire, on repasse la barre en mode normal :

In [ ]:
# Ajouter dans main.py (extrait)
# dans __init()__ -- class My_Game
        # -- TP 2 : Ajout d'un timer (actions) --
        self.player_timer: Timer = Timer(0.0)
        # -- TP 2 : FIN --

# dans __on_update()__ -- class My_Game
        # -- TP 2 : Désactivation des actions --
        if self.gui.active and self.player_timer.is_expired():
            self.gui.active = False
            self.gui.update()
         # -- TP 2 : FIN --

**Exécuter le programme** : la barre d'icônes doit s'afficher et se désactiver le temps du *cooldown* après un clic.
**Ne pas oublier** : **pip install arcade --user** (si besoin).

## Partie 2 — Lier le clic à l’animation d’attaque du joueur
On applique le **même principe** que pour l’animation de mort du monstre (TP 1).

### Étape 5 — Méthode pour l'attaque (`player.py`)
**Ajouter** une méthode `attack(self, mob)` avec *pass* comme seule instruction dans la classe `Entity`, `Mob`et `Player`.

### Étape 6 — Bloc d’animation d’attaque dans `Player.update()`
Place ce bloc **avant** le déplacement normal :

In [ ]:
# Au début de update() -- dans player.py (extrait de update())
# --- TP2 — Animation d’ATTAQUE ---
if self.status & PLAYER_ATTACK:
    # Sélection du bon statut pour l'affichage
    status_for_texture = 0
    base_dir = self.status & 0b11          # la direction est encodée sur 2 bits
    status_for_texture = ATTACK_DOWN + base_dir
    last_index = len(self.textures[status_for_texture]) - 1

    self.attack_tick += 1
    if self.attack_tick % self.attack_tick_rate == 0:
        self.current_texture_indice += 1

    # Cycle d'attaque terminé
    if self.current_texture_indice > last_index:
        back_to_walk = {
            ATTACK_DOWN:  WALK_DOWN,
            ATTACK_LEFT:  WALK_LEFT,
            ATTACK_RIGHT: WALK_RIGHT,
            ATTACK_UP:    WALK_UP,
        }
        self.status = back_to_walk[status_for_texture]
        self.status &= ~PLAYER_ATTACK         # enlève le flag attaque
        self.current_texture_indice = 0
        self.attack_tick = 0                  # reset tempo
        self.texture = self.textures[self.status][self.current_texture_indice]
    else:
        self.change_x, self.change_y = 0, 0   # figer pendant l'attaque
        self.texture = self.textures[status_for_texture][self.current_texture_indice]
        return
# --- TP2 : FIN ---


**<u>Question</u>** :**Expliquer** les codes suivants. On rappelle que *~* est le NOT bit à bit.

In [ ]:
# A Expliquer
if self.status & PLAYER_ATTACK:
    # Sélection du bon statut pour l'affichage
    status_for_texture = 0
    base_dir = self.status & 0b11          # 0b11 : 11 (2 en binaire)
    status_for_texture = ATTACK_DOWN + base_dir
    last_index = len(self.textures[status_for_texture]) - 1

# A Expliquer
self.status &= ~PLAYER_ATTACK         # enlève le flag attaque

## Checkpoints
- **C1** : La barre d’icônes s’affiche. Cliquer une icône → barre en **clicked** le temps du `Cooldown`, puis retour auto.
- **C2** : Au clic, le joueur joue **toutes les frames d’attaque**, est figé durant l’anim, puis revient à la marche.
- **C3** : Le flag `PLAYER_ATTACK` est retiré en fin d’anim.
- **C4** : Tous les attributs sont déclarés en `__init__`.